In [1]:
import urllib
from pyspark import *
from pyspark.sql import SQLContext, types
from pyspark.sql.types import StructType

In [2]:
sc = SparkContext(master="local", appName="test")

In [3]:
sc

<SparkContext master=local appName=test>

In [4]:
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'data_10_percent.gz')

data_file = './data_10_percent.gz'

lines = sc.textFile(data_file)

In [13]:
lines

./data_10_percent.gz MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
sqlContext = SQLContext(sc)

In [7]:
sqlContext

In [16]:
dataparse = lines.map(lambda x: x.split(","))
# dataparse.take(3)

In [9]:
schema = types.StructType([
    types.StructField("duration", types.IntegerType(), True),
    types.StructField("protocol_type", types.StringType(), True),
    types.StructField("service", types.StringType(), True),
    types.StructField("flag", types.StringType(), True),
    types.StructField("src_bytes", types.IntegerType(), True),
    types.StructField("dst_bytes", types.IntegerType(), True),
    types.StructField("interactions", types.StringType(), True),
    ])
schema

StructType(List(StructField(duration,IntegerType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dst_bytes,IntegerType,true),StructField(interactions,StringType,true)))

In [17]:
new_dataparse = dataparse.map(lambda key:(int(key[0]),key[1],key[2],key[3],int(key[4]),int(key[5]),key[-1]))

In [21]:
schema_lines = sqlContext.createDataFrame(new_dataparse, schema)
schema_lines.show(10)

+--------+-------------+-------+----+---------+---------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+-------+----+---------+---------+------------+
|       0|          tcp|   http|  SF|      181|     5450|     normal.|
|       0|          tcp|   http|  SF|      239|      486|     normal.|
|       0|          tcp|   http|  SF|      235|     1337|     normal.|
|       0|          tcp|   http|  SF|      219|     1337|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      212|     1940|     normal.|
|       0|          tcp|   http|  SF|      159|     4087|     normal.|
|       0|          tcp|   http|  SF|      210|      151|     normal.|
|       0|          tcp|   http|  SF|      212|      786|     normal.|
+--------+-------------+-------+----+---------+---------+------------+
only s